In [1]:
from config import MINIO_BUCKET_RAW, MINIO_BUCKET_PROCESSED
from etl.stages import stage1, stage2, stage3
from clients.postgres_client import get_postgres_properties
from clients.spark_client import create_spark_session


In [2]:
spark = create_spark_session()
postgres_props = get_postgres_properties()

25/05/06 10:55:07 WARN Utils: Your hostname, maxp-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/05/06 10:55:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/maxp/Work/spark-3.5.5-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/maxp/.ivy2/cache
The jars for the packages stored in: /home/maxp/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fde51e09-02ab-4226-95f2-4212f25f9086;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.postgresql#postgresql;42.7.5 in central
	found org.checkerframework#checker-qual;3.48.3 in central
:: resolution report :: resolve 816ms :: artifacts dl 36ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.checkerframework#checker-qual;3.48.3 from central in [default]
	org.postgresql#postgresql;42.7.5 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.

stage1: Преобразование html страниц в промежуточные таблицы events, distances, results и сохранение этих таблиц в minio

In [3]:
raw_paths = stage1(spark, postgres_props, MINIO_BUCKET_RAW, MINIO_BUCKET_PROCESSED)

25/05/06 10:56:29 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/05/06 10:57:16 WARN DAGScheduler: Broadcasting large task binary with size 1004.5 KiB
25/05/06 10:57:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/05/06 10:57:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/05/06 10:58:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


stage2: преобразование промежуточных таблиц events, distances, results в итоговые таблицы events, groups, participants, results и запись этих таблиц также в minio

In [4]:
processed_dfs = stage2(spark, raw_paths, MINIO_BUCKET_PROCESSED)

25/05/06 11:02:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


stage3: запись итоговых таблиц в postgres

In [5]:
stage3(spark, processed_dfs, postgres_props)

In [ ]:
spark.stop()